# Sample Calls on Working with Memories

In [ ]:
from bedrock_agentcore.memory import MemoryClient
client = MemoryClient(region_name="us-east-1")
memory_id="PokemonAgentMemory-zCM04KDWbq"
user_actor_id = "UserItsuki"
session_id = "session1"

# List available memories (Resources)

In [6]:
for memory in client.list_memories():
    print(f"Memory Arn: {memory.get('arn')}")
    print(f"Memory ID: {memory.get('id')}")
    print("--------------------------------------------------------------------")

Failed to list memories: An error occurred (ExpiredTokenException) when calling the ListMemories operation: The security token included in the request is expired


ClientError: An error occurred (ExpiredTokenException) when calling the ListMemories operation: The security token included in the request is expired

## Short term memory

### Maintain user context using short-term memory

##### Create events in short-term memory
- stores agent interactions into short-term memory
- Role can be USER, ASSISTANT, TOOL, etc

In [ ]:
client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=user_actor_id,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=session_id, #Unique id for a particular request/conversation.
    messages=[
        ("Tell me about pokemon", "USER"),
        ("Pokémon are creatures that humans, known as Pokémon Trainers, catch and train to battle each other for sport. The franchise began with video games created by Satoshi Tajiri and Ken Sugimori, launched by Nintendo, Game Freak, and Creatures in 1996. If you have specific questions or need advice on which Pokémon to use in battles, feel free to ask!", "ASSISTANT"),
        ("I need advice", "USER")
    ],
)

{'memoryId': 'PokemonAgentMemory-zCM04KDWbq',
 'actorId': 'UserItsuki',
 'sessionId': 'session1',
 'eventId': '0000001753835845000#3a71726e',
 'eventTimestamp': datetime.datetime(2025, 7, 30, 9, 37, 25, tzinfo=tzlocal()),
 'branch': {'name': 'main'}}

#### Load conversations from short-term memory

In [3]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=user_actor_id,
    session_id=session_id,
    max_results=5,
)
for event in sorted(events, key=lambda x: x["eventTimestamp"]):
    for item in event["payload"]:
        print(f"role: {item["conversational"]["role"].lower()}" )
        print(f"content: {item["conversational"]["content"]["text"]}" )
        print("--------------------------------------------------------------------")


role: user
content: Tell me about pokemon
--------------------------------------------------------------------
role: assistant
content: Pokémon are creatures that humans, known as Pokémon Trainers, catch and train to battle each other for sport. The franchise began with video games created by Satoshi Tajiri and Ken Sugimori, launched by Nintendo, Game Freak, and Creatures in 1996. If you have specific questions or need advice on which Pokémon to use in battles, feel free to ask!
--------------------------------------------------------------------
role: user
content: I need advice
--------------------------------------------------------------------
role: assistant
content: some advice
--------------------------------------------------------------------


#### Add more conversations to an existing session
- call `create_event` with the new message only
- when we call `list_events`, we will get two entries in this case, one with 3 messages, and one with one message (the newly added one)

In [ ]:
client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=user_actor_id,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=session_id, #Unique id for a particular request/conversation.
    messages=[
        # new message to be added only
        ("some advice", "ASSISTANT")
    ],
)

{'memoryId': 'PokemonAgentMemory-zCM04KDWbq',
 'actorId': 'UserItsuki',
 'sessionId': 'session1',
 'eventId': '0000001753838255000#47bd996e',
 'eventTimestamp': datetime.datetime(2025, 7, 30, 10, 17, 35, tzinfo=tzlocal()),
 'branch': {'name': 'main'}}

In [4]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=user_actor_id,
    session_id=session_id,
)
for event in sorted(events, key=lambda x: x["eventTimestamp"]):
    for item in event["payload"]:
        print(f"role: {item["conversational"]["role"].lower()}" )
        print(f"content: {item["conversational"]["content"]["text"]}" )
        print("--------------------------------------------------------------------")


role: user
content: Tell me about pokemon
--------------------------------------------------------------------
role: assistant
content: Pokémon are creatures that humans, known as Pokémon Trainers, catch and train to battle each other for sport. The franchise began with video games created by Satoshi Tajiri and Ken Sugimori, launched by Nintendo, Game Freak, and Creatures in 1996. If you have specific questions or need advice on which Pokémon to use in battles, feel free to ask!
--------------------------------------------------------------------
role: user
content: I need advice
--------------------------------------------------------------------
role: assistant
content: some advice
--------------------------------------------------------------------


#### Branching from an event
- Probably won't use it for regular conversation
- Useful if we want to observe how does the agent behavior diverges based on different conversations/scenarios

In [ ]:
# branch from the first event so that the second event added above is not
root_event_id = sorted(events, key=lambda x: x["eventTimestamp"])[0]["eventId"]

# (suppose you did not mean to ask about the weather tomorrow and want to undo
# that, and replace with a new message)
client.create_event(
    memory_id=memory_id,
    actor_id=user_actor_id,
    session_id=session_id,
    branch={"name": "DifferentRequest", "rootEventId": root_event_id},
    messages=[
        ("Tell me about pikachu!", "USER")
    ]
)

In [10]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=user_actor_id,
    session_id=session_id,
)
for event in sorted(events, key=lambda x: x["eventTimestamp"]):
    print(f"event branch: {event["branch"]}, id: {event["eventId"]}")
    for item in event["payload"]:
        print(f"role: {item["conversational"]["role"].lower()}" )
        print(f"content: {item["conversational"]["content"]["text"]}" )
        print("--------------------------------------------------------------------")

event branch: {'name': 'main'}, id: 0000001753835845000#3a71726e
role: user
content: Tell me about pokemon
--------------------------------------------------------------------
role: assistant
content: Pokémon are creatures that humans, known as Pokémon Trainers, catch and train to battle each other for sport. The franchise began with video games created by Satoshi Tajiri and Ken Sugimori, launched by Nintendo, Game Freak, and Creatures in 1996. If you have specific questions or need advice on which Pokémon to use in battles, feel free to ask!
--------------------------------------------------------------------
role: user
content: I need advice
--------------------------------------------------------------------
event branch: {'name': 'main'}, id: 0000001753838255000#47bd996e
role: assistant
content: some advice
--------------------------------------------------------------------
event branch: {'rootEventId': '0000001753835845000#3a71726e', 'name': 'DifferentRequest'}, id: 0000001753863

## Long term memory

### Save conversations and view extracted memories

In [ ]:
import time

# Note:
# Long-term memory records will only be extracted from events that are stored after the newly added strategies become ACTIVE.
# Conversations stored before a strategy is added will not appear in long-term memory.
event = client.create_event(
    memory_id=memory_id, # This is the id from create_memory or list_memories
    actor_id=user_actor_id,  # This is the identifier of the actor, could be an agent or end-user.
    session_id=session_id,
    messages=[
        ("Hi, tell me about pokemon", "USER"),
        ("""Pokémon are fictional creatures that trainers can capture, train, and battle. Each Pokémon belongs to a specific species and has unique abilities, types, and stats. Here are some key concepts related to Pokémon:

### Types
Each Pokémon has one or two types, such as Fire, Water, Grass, Electric, Psychic, and many more. Understanding these types is crucial for battles, as certain types have strengths and weaknesses against others. For example:
- Fire is strong against Grass but weak against Water.
- Water is strong against Fire but weak against Electric.

### Statistics
Pokémon have various statistics, including:
- **HP (Hit Points)**: Determines how much damage a Pokémon can take.
- **Attack**: Influences the damage of physical moves.
- **Defense**: Reduces damage taken from physical moves.
- **Special Attack** and **Special Defense**: Affect damage from special moves and resistance to them.
- **Speed**: Determines the order of action in battles.

### Abilities and Moves
Each Pokémon has an ability that provides passive effects in battles. Moves can be attacks, status changes, or support techniques. Pokémon can learn new moves as they level up or through TM (Technical Machines) and HM (Hidden Machines) items.

### Evolution
Many Pokémon can evolve into more powerful forms when they reach certain levels, meet specific conditions, or use special items. Evolution often changes a Pokémon's type and boosts its stats.

### Breeding and Training
Trainers can breed Pokémon to produce offspring with inherited moves and abilities. Training involves leveling up Pokémon to make them stronger.

### Competitive Play
In competitive battling, players build teams considering type match-ups, move sets, and strategies to outsmart opponents.

If you have specific questions about certain Pokémon, types, or strategies, feel free to ask!""", "ASSISTANT"),

        ("Help me choose one battleing and give me some advice on strategies to take", "USER"),
        ("""Pokémon Suggestion: **Charizard** (Fire/Flying Type)
Charizard is versatile and can be a strong choice for many situations.
#### Strategies:
1. **Moveset**:
   - **Fire Blast / Flamethrower**: Powerful Fire-type moves. Fire Blast is stronger but has lower accuracy; Flamethrower is reliable.
   - **Air Slash**: A strong Flying-type move that can flinch opponents.
   - **Dragon Pulse**: Useful against Dragon-types.
   - **Roost**: Healing move to recover HP, keeping Charizard longer in battle.

2. **Ability**:
   - **Blaze**: Boosts Fire-type moves when HP is low. Useful for surprise KO risks.
   - Alternatively, consider **Solar Power** if you have access to sunny weather (like with Sunny Day). It boosts Special Attack but lowers HP each turn.

3. **Item**:
   - **Charizardite X/Y** (Mega Evolution): Depending on your preferred form. Mega Charizard X becomes a Fire/Dragon type with enhanced physical attack, while Mega Charizard Y boosts its Special Attack further with Drought.
   - **Choice Specs**: Boosts Special Attack but locks you into one move until you switch.
...
- **Switch Tactics**: Don’t hesitate to switch out Charizard if the matchup is unfavorable. Keeping your Pokémon healthy is crucial.
- **Prediction**: Try to predict your opponent's moves and switch to a Pokémon that resists that type.

Feel free to share your preferences or current Pokémon, and I can provide more tailored advice!""",
            "ASSISTANT",
        ),
        ("what if I want to use pikachu", "USER"),
        ("""If you want to use **Pikachu** in battle, that can be a fun choice! Pikachu is an Electric-type Pokémon known for its speed and special attack. Let’s explore strategies, movesets, and tips to maximize its effectiveness!

### Pikachu Overview
**Type**: Electric
**Base Stats**:
- HP: 35
- Attack: 55
- Defense: 40
- Special Attack: 50
- Special Defense: 50
- Speed: 90

### Strengths
- **Type Advantage**: Strong against Water and Flying types.
- **Speed**: Fast enough to outspeed many Pokémon, allowing you to strike first.

### Recommended Moveset
1. **Thunderbolt**: Reliable Electric-type move with decent power and good accuracy.
2. **Volt Tackle**: High-power Electric move that causes recoil damage to Pikachu. Great for finishing off opponents.
3. **Grass Knot**: Covers Ground types, which resist Electric moves.
4. **Nuzzle**: A move that not only deals damage but also paralyzes the opponent.

### Ability
- **Static**: May paralyze opponents who make physical contact, providing additional utility.
...
- Teach Pikachu moves that can help it switch in safely, like Volt Switch, which allows for quick escapes while dealing damage.
- Use Pikachu on the offensive primary during the mid-to-late game when opposing Pokémon are weakened.

Overall, while Pikachu may not be as powerful as some fully evolved Pokémon, using it creatively can surprise opponents and lead to victories! Let me know if you need more details or have any specific battles in mind!""", "ASSISTANT"),
    ],
)

# long-term memory is asynchronous so we will need to
# Wait for meaningful memories to be extracted from the conversation.
time.sleep(60)


In [26]:
# Query for the summary of the issue using the namespace set in summary strategy above
memories = client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"/summaries/{user_actor_id}/{session_id}",
    query="can you summarize the battle strategies?",
    top_k = 3
)

for memory in sorted(memories, key=lambda x: x["createdAt"]):
    print(f"created at: {memory["createdAt"]}")
    print(f"score: {memory["score"]}")
    print(f"content: {memory["content"]["text"]}")
    print("--------------------------------------------------------------------")


created at: 2025-07-31 08:30:15+09:00
score: 0.42846677
content: <summary>
    <topic name="Pokémon Basics">
        Pokémon are fictional creatures that trainers can capture, train, and battle. Each Pokémon has specific species characteristics, abilities, types, and stats. Key concepts include types (like Fire, Water, Grass, Electric, Psychic) which determine strengths and weaknesses in battles, statistics (HP, Attack, Defense, Special Attack, Special Defense, Speed), abilities that provide passive battle effects, moves for attacking and support, evolution into more powerful forms, breeding and training mechanics, and competitive play strategies.
    </topic>
    <topic name="Charizard Battle Strategy">
        Charizard is a Fire/Flying type Pokémon recommended for battling. Suggested moveset includes Fire Blast/Flamethrower (for power vs. reliability), Air Slash (for flinch chance), Dragon Pulse (against Dragon types), and Roost (for healing). Ability options include Blaze (boosts F